In [ ]:
import os
import pandas as pd
import numpy as np

In [ ]:
sample_open_data_ath = "E:\gc_opendata\9787e81f-19f6-4cc1-891d-63e68b4d2f9f"
athlete_files = next(os.walk(sample_open_data_ath), (None, None, []))[2]
# [athlete_files.remove(file) if '.csv' not in file else None for file in athlete_files]
# athlete_activity_filepaths = [os.path.join(sample_open_data_ath,filepath) for filepath in athlete_files]

In [ ]:
with open(os.path.join(sample_open_data_ath,'{9787e81f-19f6-4cc1-891d-63e68b4d2f9f}.json'),'r') as f:
    ath_over = f.read()
f.close()

In [ ]:
import json

In [ ]:
a = json.loads(ath_over)

In [ ]:
a['RIDES'][0]['METRICS'].keys()

In [ ]:
a['RIDES'][500]['METRICS']['cp_setting']

In [ ]:
class athlete(object):
    def __init__

In [ ]:
def calc_xspeed(x, secs_id, dist_id, elev_id):
    xspeed = (
        1+9*(                                  # slope adj
            (x[elev_id,1:] - x[elev_id,:-1]) / # delta alt
            (x[dist_id,1:] - x[dist_id,:-1])   # delta dist
        )
    ) * (
            (x[dist_id,1:] - x[dist_id,:-1]) / # delta dist
            (x[secs_id,1:] - x[secs_id,:-1])   # delta secs
            * (60*60)                          # convert to kph
        )                                      # slope running speed
    return xspeed

def extract_activity(filepath, metrics, make_relative=True, make_xspeed=False):
    df = pd.read_csv(filepath, engine='pyarrow')
    activity_array = df[metrics].to_numpy().T
    if make_xspeed:
        xspeed = calc_xspeed(activity_array, metrics.index('secs'), metrics.index('km'), metrics.index('alt'))
        xspeed = np.concatenate((np.array([0]),xspeed))
        activity_array = np.concatenate((activity_array,[xspeed]), axis=0)
    return activity_array

In [ ]:
filepath = athlete_activity_filepaths[100]
metrics = ['power','hr','km','alt','secs']
extract_activity(filepath, metrics, make_xspeed=True)


In [ ]:
import torch
import torch.nn as nn

In [ ]:
class Net(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super().__init__()
        self.input = nn.Linear(in_features=input_dim, out_features=hidden_dim)
        self.hidden_1 = nn.Linear(in_features=hidden_dim, out_features=hidden_dim)
        self.output = nn.Linear(in_features=16, out_features=output_dim)
        
    def forward(self, x):
        x = F.relu(self.input(x))
        x = F.relu(self.hidden_1(x))
        return self.output(x)
    
    
model = Net()
print(model)

In [ ]:
num_epochs = 200
train_accuracies, test_accuracies = [], []

loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)

for epoch in range(num_epochs):
    # Train set
    for X, y in train_loader:
        preds = model(X)
        pred_labels = torch.argmax(preds, axis=1)
        loss = loss_function(preds, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    train_accuracies.append(
        100 * torch.mean((pred_labels == y).float()).item()
    )
    
    # Test set
    X, y = next(iter(test_loader))
    pred_labels = torch.argmax(model(X), axis=1)
    test_accuracies.append(
        100 * torch.mean((pred_labels == y).float()).item()
    )